In [10]:
import sys
import os

try:
    from diffform import *
except ImportError as e:
    sys.path.append(os.getcwd()+'/..')
    from core import *


from sympy import symbols, sin, Function, I, Array, permutedims, Eq, solve, cos, simplify, sqrt

set_max_degree(4)

coords = t,r,theta,phi = symbols(r"t r \theta \phi")
basis = dt,dr,dtheta,dphi = d(t), d(r), d(theta), d(phi)

# f = sqrt(1-1/r)
f = Function("f")(r)

# g = 1/f
g = Function("g")(r)

e0 = f*dt
e1 = g*dr
e2 = r*dtheta
e3 = r*sin(theta)*dphi

display("Tetrads")
display(e0)
display(e1)
display(e2)
display(e3)
print("")

S1 = I*e0*e1 - e2*e3
S2 = I*e0*e2 - e3*e1
S3 = I*e0*e3 - e1*e2

AS1 = I*e0*e1 + e2*e3
AS2 = I*e0*e2 + e3*e1
AS3 = I*e0*e3 + e1*e2

display("Self-Dual 2-forms")
display(S1)
display(S2)
display(S3)

A_symbols = permutedims(Array([symbols(f"A^{{1:4}}_{{{str(c)}}}") for c in coords]),(1,0))

A1 = sum([A_symbols[0,i]*basis[i] for i in range(4)])
A2 = sum([A_symbols[1,i]*basis[i] for i in range(4)])
A3 = sum([A_symbols[2,i]*basis[i] for i in range(4)])

# A1 = I*f.diff(r)/g*dt + cos(theta)*dphi 
# A2 = -1/g * sin(theta) * dphi
# A3 = 1/g * dtheta

dAS1 = S1.d + A2*S3 - A3*S2
dAS2 = S2.d + A3*S1 - A1*S3
dAS3 = S3.d + A1*S2 - A2*S1

dA_equations = [Eq(fa,0) for fa in dAS1.factors] + [Eq(fa,0) for fa in dAS2.factors] + [Eq(fa,0) for fa in dAS3.factors]

A_sol = solve(dA_equations)

A_solution = A_symbols.subs(A_sol[-1])

A1 = sum([A_solution[0,i]*basis[i] for i in range(4)])
A2 = sum([A_solution[1,i]*basis[i] for i in range(4)])
A3 = sum([A_solution[2,i]*basis[i] for i in range(4)])

print("")
print("Self-Dual Connection")
display(A1)
display(A2)
display(A3)

F1 = A1.d + A2*A3
F2 = A2.d + A3*A1
F3 = A3.d + A1*A2

F  = [F1,F2,F3]
S  = [S1,S2,S3]
AS = [AS1,AS2,AS3]

psi_symbols = Array([[symbols(rf"\psi^{{{i+1}{j+1}}}") for j in range(3)] for i in range(3)])
R_symbols = Array([[symbols(rf"R^{{{i+1}{j+1}}}") for j in range(3)] for i in range(3)])

psi_R_symbols = []
for i in range(3):
    for j in range(3):
        psi_R_symbols.append(psi_symbols[i,j])
        psi_R_symbols.append(R_symbols[i,j])

F_PS_RAS = [ F[i] - sum([ psi_symbols[i,j]*S[j] + R_symbols[i,j]*AS[j] for j in range(3)]) for i in range(3)]

psi_R_equations = []
for ps in F_PS_RAS:
    for  fa in ps.factors:
        psi_R_equations.append(Eq(fa,0))
    
psi_R_sols = solve(psi_R_equations,psi_R_symbols)

R_solution   = R_symbols.subs(psi_R_sols)
psi_solution = psi_symbols.subs(psi_R_sols)

print("")
print("Self-Dual Weyl")
display(psi_solution)
print("Self-Dual Ricci")
display(R_solution)

backup import


'Tetrads'

'Self-Dual 2-forms'


Self-Dual Connection



Self-Dual Weyl


[[-Derivative(f(r), (r, 2))/(2*f(r)*g(r)**2) + Derivative(f(r), r)*Derivative(g(r), r)/(2*f(r)*g(r)**3) + 1/(2*r**2) - 1/(2*r**2*g(r)**2), 0, 0], [0, Derivative(g(r), r)/(2*r*g(r)**3) - Derivative(f(r), r)/(2*r*f(r)*g(r)**2), 0], [0, 0, Derivative(g(r), r)/(2*r*g(r)**3) - Derivative(f(r), r)/(2*r*f(r)*g(r)**2)]]

Self-Dual Ricci


[[-Derivative(f(r), (r, 2))/(2*f(r)*g(r)**2) + Derivative(f(r), r)*Derivative(g(r), r)/(2*f(r)*g(r)**3) - 1/(2*r**2) + 1/(2*r**2*g(r)**2), 0, 0], [0, -Derivative(g(r), r)/(2*r*g(r)**3) - Derivative(f(r), r)/(2*r*f(r)*g(r)**2), 0], [0, 0, -Derivative(g(r), r)/(2*r*g(r)**3) - Derivative(f(r), r)/(2*r*f(r)*g(r)**2)]]

In [11]:
import sys
import os

try:
    from diffform import *
except ImportError as e:
    sys.path.append(os.getcwd()+'/..')
    from core import *

from sympy import symbols, sin, cos, Matrix, simplify

coords = theta,phi = symbols(r"\theta \phi")
basis = dtheta,dphi = d(theta), d(phi)

x = cos(theta)**2*dtheta
y = sin(theta)**2*dtheta

g = Matrix([[cos(theta),sin(theta)],[-sin(theta),cos(theta)]])